# HOMEWORK 1
This is my first Project which was given to me as an exercise for the Theoretical Astrophysics II: Polarized Radiative Transfer course at the university of Freiburg. The goal is to synthesize a spectral line profile in Local Thermodynamic Equilibrium from a given atmospheric model. 

To do this I am going to calculate level populations using the Boltzmann and Saha equations, compute line opacity, source function and solve the radiative transfer equation.

The atmospheric model is provided by the "falc.dat", which can be found in this repository.

In [ ]:
#all needed imports
import numpy as np
import matplotlib.pyplot as plt
import scipy

#data
continuum_optical_depth_500, height, temperature, gas_pressure, electron_pressure = np.loadtxt('falc_71.txt', usecols=(0,1,2,3,4), unpack=True)

<br><br>
### STEP 1: Solve for the ionization state

As a warm-up the Saha equation (only for Hydrogen) is used to compute the ionization state of Hydrogen as a funktion of height in the atmosphere. We will get the electron density and compare it with the one given in the model.

The Saha equation is given by:
$\frac{n_{j+1}n_e}{n_j} = \frac{2Z_{j+1}}{Z_j} \left(\frac{2 \pi m_e k T}{h^2} \right)^{\frac{3}{2}}e^{-\frac{E_j}{kT}}$